In [46]:
!pip install beautifulsoup4


In [47]:
from bs4 import BeautifulSoup
soup = BeautifulSoup("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", 'html.parser')

print(soup.prettify())

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M



In [48]:
import requests
import pandas as pd
# Set headers
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})


In [49]:


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())




<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XhtmlwpAMFUAAC-M7LMAAACK","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":933624196,"wgRevisionId":933624196,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

# FINDING HTML ELEMENTS AND EXTRACTING DATA

In [50]:
table=soup.find('table', attrs={"class": "wikitable"}) 
table_extract = table.find_all('tr')

l=['Postal Code','Borough','Neighborhood']
p=pd.DataFrame(columns=l)
for item in table_extract:
    rows = item.contents
    if(rows[1].text not in p['Postal Code'].values.tolist()):
        if(rows[5].text.replace('\n','')=='Not assigned'):
            c=rows[3].text
        else:
            c=rows[5].text
        if(len(rows)>3):
            if(rows[3].text!='Not assigned'):
                p = p.append({'Postal Code':rows[1].text,'Borough':rows[3].text,'Neighborhood':c.replace('\n','')}, ignore_index=True)
    else:
        i=p.index[p['Postal Code']==rows[1].text]
        p.iloc[i[0]]['Neighborhood']+=','+rows[5].text.replace('\n','')

p=p.drop(0)
p


,Postal Code,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,"Rouge,Malvern"
8,M3B,North York,Don Mills North
9,M4B,East York,"Woodbine Gardens,Parkview Hill"
10,M5B,Downtown Toronto,"Ryerson,Garden District"


In [51]:
p.shape

(103, 3)

In [52]:
!pip install geocoder

In [53]:
p['Postal Code'].values[0]

'M3A'

In [54]:
import geocoder # import geocoder
lat=[]
long=[]
# initialize your variable to None
lat_lng_coords = None
pc=p['Postal Code'].values
b=p['Borough'].values
n=p['Neighborhood'].values
df = pd.read_csv("http://cocl.us/Geospatial_data")
p=pd.merge(p, df, on='Postal Code')
p

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


# taking out only those boroghs that have Toronto in them


In [65]:
p=p[p['Borough'].str.contains('Toronto')]
p.reset_index(drop=True)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
9,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259


In [77]:
CLIENT_ID = '3WTXZHO4BP1MLZP3SJAVCOZ55UKBESCA35CPTEZJUMXVHS15' # your Foursquare ID
CLIENT_SECRET = 'XHUG4UW2EOWQF0C2ZLLGP3BHPMHUI4XZRIR534BEWSQJ5DJX' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT=500

Your credentails:
CLIENT_ID: 3WTXZHO4BP1MLZP3SJAVCOZ55UKBESCA35CPTEZJUMXVHS15
CLIENT_SECRET:XHUG4UW2EOWQF0C2ZLLGP3BHPMHUI4XZRIR534BEWSQJ5DJX


In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:

venues = getNearbyVenues(names=p['Borough'],
                                   latitudes=p['Latitude'],
                                   longitudes=p['Longitude']
                                  )
venues

Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.654260,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Downtown Toronto,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,Downtown Toronto,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
6,Downtown Toronto,43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
7,Downtown Toronto,43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
8,Downtown Toronto,43.654260,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
9,Downtown Toronto,43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [103]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
latitude=43.6532
longitude=79.3832
# import k-means from clustering stage
from sklearn.cluster import KMeans
import numpy as np
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# set number of clusters
# one hot encoding
toronto_venues=venues
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
print(toronto_grouped)
kclusters = 2
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)
toronto_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] # add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = p

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

            Borough  Afghan Restaurant   Airport  Airport Food Court  \
0   Central Toronto           0.000000  0.000000            0.000000   
1  Downtown Toronto           0.000761  0.000761            0.000761   
2      East Toronto           0.000000  0.000000            0.000000   
3      West Toronto           0.000000  0.000000            0.000000   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0        0.000000         0.000000          0.000000             0.018182   
1        0.001522         0.002283          0.001522             0.011416   
2        0.000000         0.000000          0.000000             0.023810   
3        0.000000         0.000000          0.000000             0.000000   

   Antique Shop  Aquarium  ...  Toy / Game Store     Trail  Train Station  \
0      0.000000  0.000000  ...          0.009091  0.009091       0.000000   
1      0.001522  0.003805  ...          0.000761  0.000761       0.002283   
2      0.000000  0.000

# we have now clustered boroughs 